# __THƯ VIỆN__

In [ ]:
import pandas as pd
import torch

from PIL import Image
import os
import io
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet

import string
import re
import nltk
import json
nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

import ast

# from vietocr.tool.predictor import Predictor
# from vietocr.tool.config import Cfg

# __DỮ LIỆU__

In [ ]:
folder = 'E:\\UIT DATA SCIENCE\\data samples\\'

In [ ]:
file_name = folder + 'vimmsd-private-test.json'

with open(file_name, 'r', encoding='utf-8') as f:
    data = json.load(f)

df_private = pd.DataFrame(data)
df_private = df_private.transpose()
df_private

In [ ]:
file_name = folder + 'vimmsd-public-test.json'

with open(file_name, 'r', encoding='utf-8') as f:
    data = json.load(f)

df_public = pd.DataFrame(data)
df_public = df_public.transpose()
df_public

In [150]:
df_images = pd.read_excel(folder + 'file_train.xlsx')
df_images

,Label,image
0,NaN,8ae451edcd8ebf697f8763ece249115813149c55733bf8...
1,CALZ Chốn Sukinchana bàn thờ đột nhiên bị đổ r...,35370ffd6c791d6f8c4ab3dd4363ed468fab41e4824ee9...
2,NaN,316fdd1477725b9fb1a55015ac06b68b92b50bd4303e08...
3,SHARK VIỆT: ?NHIỀU NGƯỜI Ở VIỆT NAM ĐI LÀM CHỈ...,8a0f34e0e30e4e5cfb306933c1d25fa801a5da78646b59...
4,NaN,e517a5e95d1065886a7c815e82fe254381d4f9f4b244d4...
...,...,...
10800,LỘN ĐẦU RỒI,46ce5ad52085691fc81869c82e8222c0d737b34fedc2bd...
10801,NaN,f816f7152cca9c5899f818ce681cf0949c6964ea2fc5ae...
10802,"ĐƯỢC BIẾT, CHIẾC MÁY CÁC NHÀ KHẢO ẢNH NÀY ĐÃ C...",0accae8d37f9edc90b5f0a2f5f3cca773f5d01b5124302...
10803,TỚ SẼ DÙNG CÂY VỚI MỘT BÚA NÀY ĐẬP NÁT TAY THÔ...,bf125e295f85c0946940b789b2ba10106b2a85b9e70d88...


In [151]:
df_train = df_train.merge(df_images[['image', 'Label']], on='image', how='left')
df_train.rename(columns={'Label': 'image_text'}, inplace=True)
df_train

,image,caption,label,image_text
0,8ae451edcd8ebf697f8763ece249115813149c55733bf8...,Cô ấy trên mạng vs cô ấy ngoài đời =))),multi-sarcasm,NaN
1,35370ffd6c791d6f8c4ab3dd4363ed468fab41e4824ee9...,Người tâm linh giao tiếp với người thực tế :))),not-sarcasm,CALZ Chốn Sukinchana bàn thờ đột nhiên bị đổ r...
2,316fdd1477725b9fb1a55015ac06b68b92b50bd4303e08...,Hình như Trăng hôm nay đẹp quá mọi người ạ! 😃 ...,multi-sarcasm,NaN
3,8a0f34e0e30e4e5cfb306933c1d25fa801a5da78646b59...,MỌI NGƯỜI NGHĨ SAO VỀ PHÁT BIỂU CỦA SHARK VIỆT...,not-sarcasm,SHARK VIỆT: ?NHIỀU NGƯỜI Ở VIỆT NAM ĐI LÀM CHỈ...
4,e517a5e95d1065886a7c815e82fe254381d4f9f4b244d4...,2 tay hai nàng chứ việc gì phải lệ hai hàng,multi-sarcasm,NaN
...,...,...,...,...
10800,46ce5ad52085691fc81869c82e8222c0d737b34fedc2bd...,Lộn đầu rồi,not-sarcasm,LỘN ĐẦU RỒI
10801,f816f7152cca9c5899f818ce681cf0949c6964ea2fc5ae...,"Chào các bạn, mình là Goda Takeshi. Trong live...",not-sarcasm,NaN
10802,0accae8d37f9edc90b5f0a2f5f3cca773f5d01b5124302...,Cre: Hùynh Quốc Huy,not-sarcasm,"ĐƯỢC BIẾT, CHIẾC MÁY CÁC NHÀ KHẢO ẢNH NÀY ĐÃ C..."
10803,bf125e295f85c0946940b789b2ba10106b2a85b9e70d88...,Anh hùng thật sự,not-sarcasm,TỚ SẼ DÙNG CÂY VỚI MỘT BÚA NÀY ĐẬP NÁT TAY THÔ...


## __CHUẨN  BỊ DỮ LIỆU ĐỂ TRAIN__

### __DÙNG CLIP__

#### __IMAGES__

In [6]:
from transformers import CLIPProcessor, CLIPModel

# Tải mô hình và processor từ Hugging Face
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

c:\Users\Nguyen_Thanh_Vinh\anaconda3\envs\RP_Env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Nguyen_Thanh_Vinh\anaconda3\envs\RP_Env\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
def vector_image(data):

    count = 0

    train_X = pd.DataFrame(columns=['image', 'image_vector', 'sarcasm'])
    for idx, row in data.iterrows():

        try:
            image = folder + 'test-images\\' + row['image']

            image = Image.open(image)

            inputs = processor(images=image, return_tensors="pt")

            # Trích xuất đặc trưng (embedding) từ hình ảnh
            with torch.no_grad():
                image_features = model.get_image_features(**inputs)

            # text = row['text']
            # inputs_text = processor(text=text, return_tensors="pt")

            # # Trích xuất đặc trưng (embedding) từ văn bản
            # with torch.no_grad():
            #     text_features = model.get_text_features(**inputs_text)

            new_row = {'image': row['image'], 'image_vector': image_features, 'sarcasm': row['label']}
            train_X.loc[len(train_X)] = new_row
        except Exception as e:
            print(row['image'])

        if int(idx) == len(data) - 1:
            count += 1
            file_name = f'image_private_{count}.csv'
            train_X.to_csv(file_name, index=False)
            print('Save {}'.format(count))
            train_X = pd.DataFrame(columns=['image', 'image_vector', 'sarcasm'])

In [11]:
vector_image(df_private)

Save 1


#### __CAPTION__

In [13]:
def vector_text(data):
    count = 0
    train_X = pd.DataFrame(columns=['id', 'caption_features', 'sarcasm'])

    for idx, row in data.iterrows():
        try:
            text = row['caption']
            # Mã hóa văn bản và lấy số lượng token
            inputs = processor(text=text, return_tensors="pt", truncation=False)
            token_ids = inputs['input_ids'][0]  # Lấy tensor đầu tiên

            # Kiểm tra số lượng token
            if len(token_ids) > 77:
                # Chia nhỏ thành các đoạn không quá 76 token
                chunks = [token_ids[i:i + 76] for i in range(0, len(token_ids), 76)]
            else:
                chunks = [token_ids]

            text_features_list = []

            # Xử lý từng đoạn
            for chunk in chunks:
                inputs_chunk = processor.tokenizer.decode(chunk, skip_special_tokens=True)
                inputs = processor(text=inputs_chunk, return_tensors="pt", truncation=True, padding='max_length', max_length=77)

                # Lấy đặc trưng văn bản
                with torch.no_grad():
                    text_features = model.get_text_features(**inputs)
                    text_features_list.append(text_features)

            # Kết hợp các đặc trưng
            if text_features_list:
                text_features_tensor = torch.stack(text_features_list)
                combined_features = torch.mean(text_features_tensor, dim=0)

                new_row = {'id': idx, 'caption_features': combined_features, 'sarcasm': row['label']}
                train_X.loc[len(train_X)] = new_row

        except Exception as e:
            print(f"Error processing row {idx}: {e}")

        if int(idx) == len(data) - 1:
            count += 1
            file_name = f'test_private_{count}.csv'
            train_X.to_csv(file_name, index=False)
            print(f'Save {count}')
            train_X = pd.DataFrame(columns=['id', 'caption_features', 'sarcasm'])


In [14]:
vector_text(df_private)

Token indices sequence length is longer than the specified maximum sequence length for this model (271 > 77). Running this sequence through the model will result in indexing errors


Save 1


#### __IMAGE_TEXT__

In [16]:
df = pd.read_excel(folder + 'private_text_in_image.xlsx')
df

,Label,image
0,LINVE,066d6021fdfeaf39f1dec523879e8fe4d35e877abcea44...
1,Wileup,555f4787d4df49e7be743b3d5b77c90755f0d6c351f36b...
2,AMANYMS Brizm LINVE,7b7cdea2cde1f3f93371259b587a03f2e8c0af682b4d51...
3,Brizm LINVE,80167e59d729cf3aaba5d2d3da40db6995cb8a6a8c4a88...
4,m 12 GNOGASA ROUP FAN MUSẼ THẤY THẬT LÀ CẢM GI...,59db087307031d60755af3a5c01a44ba55a04bfab21027...
...,...,...
1499,Thắng Cuội 1 giờ Mẹ bảo hai đứa nên đi tập gym...,3c643826258f8aacc8a98d8e24956f909797010f1e80bd...
1500,"A/ KFC] TỪ ĂN LẤY HÊN ĐI CON, COMBO LỘC PHÚC V...",09f5adf3e555d3066eae0be356e5ce797c066706e7c808...
1501,"BÉO [RA RỒI SUỐT NGÀY ĂN VỚI NGỦ, PHẢI KIỂM SO...",b11515c1aa521da4f6d0cd6464e5ea3e030662cdd43f7f...
1502,24 09638222222 Mediavtv24eagmall.com 76% số ca...,8a83634808704a5c7493327893f793effd6e78cb037ac4...


In [19]:
def vector_text(data):
    count = 0
    train_X = pd.DataFrame(columns=['image', 'image_text'])

    for idx, row in data.iterrows():
        try:
            text = row['Label']
            # Mã hóa văn bản và lấy số lượng token
            if pd.isna(text):  # Kiểm tra nếu text là NaN
                # Gán vector toàn số 0
                zero_vector = torch.zeros(1, model.config.projection_dim)  # Kích thước vector theo mô hình
                new_row = {'image': row['image'], 'image_text': zero_vector}
                train_X.loc[len(train_X)] = new_row
                continue  # Bỏ qua phần còn lại và chuyển sang hàng tiếp theo
            
            inputs = processor(text=text, return_tensors="pt", truncation=False)
            token_ids = inputs['input_ids'][0]  # Lấy tensor đầu tiên

            # Kiểm tra số lượng token
            if len(token_ids) > 77:
                # Chia nhỏ thành các đoạn không quá 76 token
                chunks = [token_ids[i:i + 76] for i in range(0, len(token_ids), 76)]
            else:
                chunks = [token_ids]

            text_features_list = []

            # Xử lý từng đoạn
            for chunk in chunks:
                inputs_chunk = processor.tokenizer.decode(chunk, skip_special_tokens=True)
                inputs = processor(text=inputs_chunk, return_tensors="pt", truncation=True, padding='max_length', max_length=77)

                # Lấy đặc trưng văn bản
                with torch.no_grad():
                    text_features = model.get_text_features(**inputs)
                    text_features_list.append(text_features)

            # Kết hợp các đặc trưng
            if text_features_list:
                text_features_tensor = torch.stack(text_features_list)
                combined_features = torch.mean(text_features_tensor, dim=0)

                new_row = {'image': row['image'], 'image_text': combined_features}
                train_X.loc[len(train_X)] = new_row

        except Exception as e:
            print(f"Error processing row {idx}: {e}")

        if int(idx) == len(data) - 1:
            count += 1
            file_name = f'image_text_private_{count}.csv'
            train_X.to_csv(file_name, index=False)
            print(f'Save {count}')
            train_X = pd.DataFrame(columns=['image', 'image_text'])


In [20]:
vector_text(df)

Save 1
